L2CS-Net의 input은 
1. 얼굴 부분 이미지 (나중에 모델의 dataset class가 448x448로 resize 하므로 크기 상관 없음)
2. 3D Gaze 각도
3. 2D Gaze 각도 

각도 범위 180도 이내 조건 있은 확인 필요 (아마 2D에서만?) 

# To do
pose 정보와 distance 정보들로 3D Gaze, 2D Gaze 계산

그 외에는 일단 돌아갈 듯 함. L2CS-Net 모델링은 이걸로 끝? 

In [67]:
import os
import numpy as np
import torch
from torch.utils.data.dataset import Dataset
from PIL import Image
from glob import glob
import random
import json
import cv2
import matplotlib.pyplot as plt 
from pathlib import Path
import nia22
from nia22 import utils
import shutil

Label  
FORMAT:  
Face left right origin 3DGaze 2DGaze

In [2]:
def cut_face(img, eye, factor = 1.5):
    # factor = 1.5: 눈 사이 거리의 +/- 1.5배를 얼굴로 봄
    both_eyes = np.array(eye.l_eyelid['points']+eye.r_eyelid['points'])

    center = both_eyes.mean(axis=0).astype(int)

    ll = np.array(eye.l_eyelid['points'])
    rr = np.array(eye.r_eyelid['points'])

    lc = np.mean(ll, axis=0)
    rc = np.mean(rr, axis=0)

    d_eyes = (np.sqrt(np.sum(np.square(lc - rc))) * factor).astype(int)

    # make sure square and within the image boundary
    lower = center - d_eyes

    lower[lower < 0] = 0

    upper = np.array([min((1920, lower[0] + 2*d_eyes)), min((1080, lower[1] + 2*d_eyes))])

    aside = min((upper - lower))

    upper[0] = lower[0] + aside
    upper[1] = lower[1] + aside
    
    return img[lower[1]:upper[1], lower[0]:upper[0]]

얼굴 crop  

얼굴 = 양눈 중앙에서 위-아래로 양눈 거리 * n 만큼

양 눈의 dx보다 dy가 더 큰 경우 = 이미지가 옆으로 누워있음. 


### mkdir

In [3]:
# Load annotation
base_dir = Path("/home/hoseung/Work/NIA/") 
json_dir = base_dir / Path("data/json/")
raw_dir = base_dir / Path("data/raw/")

# output dir
label_dir = base_dir / Path("data/l2cs_data/Label/")
image_dir = base_dir / Path("data/l2cs_data/Image/")

os.makedirs(label_dir, exist_ok=True)
os.makedirs(image_dir, exist_ok=True)
for this_dir in [label_dir, image_dir]:
    os.makedirs(this_dir / Path("test/"), exist_ok=True)
    os.makedirs(this_dir / Path("train/"), exist_ok=True)
    os.makedirs(this_dir / Path("valid/"), exist_ok=True)

cnt = 0

In [4]:
# dkey <-> device
conditions = ["F", "S", "D", "A", "N"]
postures = ["S", "D", "P", "L", "F", "C", "H", "E", "T", "U"]
device = ["S", "T", "L", "V", "M"]

dir_names = ["Eye-tracker/", "FaceAngle/", 
             "CamAngle/", "DistCam2Face/", 
             "DistDisp2Face/"]
file_names = ["_point_", "_head_", "_cam_", 
              "_dcam_", "_ddisp_"]

In [5]:
vl = utils.VidLoader(base_dir)

Base dir: /home/hoseung/Work/NIA/data/raw


In [ ]:
def move_files(fn_label, jlabel, prefix):
    with open(fn_label, "w") as f:
        f.write(header)
        for ll in jlabel:
            fn_old = ll.split(" ")[0]
            ll = prefix+ll
            fn_new = ll.split(" ")[0]
            f.write(ll+'\n')
            
            shutil.move(image_dir / Path(fn_old), image_dir / Path(fn_new))    


In [87]:
simage_dir = str(image_dir) + '/'
with open(label_dir / Path(f"all.label"), "w") as f:
    f.write("Face Origin 2DGaze\n")
    cnt = 0
    for uid in range(131, 135):
        for device in ["L", "M", "V", "T", "S"][:3]:
            vlist = vl.load(uid, device=device)
            # save PNG 
            for fn_vid in vlist:
                fn_base = fn_vid.split("/")[-1].split(".mp4")[0]
                # load rpy
                fn_rpy = fn_vid.replace(".mp4", ".txt").replace("RGB/", "rpy/").replace("_rgb_", "_rpy_")
                dd = pd.read_csv(fn_rpy, delim_whitespace=True, skiprows=1, usecols=[0,1,2,3,4,5],
                                names=['pitch', 'yaw', 'xmin', 'ymin', 'xmax', 'ymax'])
                
                cap = cv2.VideoCapture(fn_vid)
                if cap.isOpened():
                    jlist = vl.get_jsons(fn_vid)                
                    for ff, fn_json in zip(jlist['frames'], jlist['jsons']):
                        #if True:
                        try:
                            #anno = json.load(open(fn_json,"r"))
                            cap.set(cv2.CAP_PROP_POS_FRAMES, ff)
                            ok, frame = cap.read()
                            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                            
                            #eye = nia22.eyes.Eye(anno["Annotations"]["annotations"])
                            #cropped = cut_face(frame, eye)
                            pitch, yaw, ymin, xmin, ymax, xmax = dd.iloc[ff]
                            cropped = frame[int(xmin):int(xmax),
                                            int(ymin):int(ymax)]

                            # get 2D gaze from rpy 
                            #gaze2d = calculate_gaze2d(anno)
                            # save in Image and Label directory
                            fn_png = fn_json.replace(".json", ".png")
                            fn_org = fn_png.split("/home/hoseung/Work/NIA/data/")[1]
                            fn_face = f"Face/{cnt:06d}.jpg"
                            
                            f.write(f"{fn_face} {fn_org} {pitch:.3f},{yaw:.3f}\n")

                            cropped = cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB)
                            cv2.imwrite(simage_dir + fn_face, cropped)
                        except:
                        #else:
                            print("something wrong", fn_json)
                        cnt += 1
                else:
                    print("Can't open video file")
                    print(">>>", fn_vid)
                cap.release()
                print(fn_vid, "done")


# Read all.label
with open(label_dir / Path(f"all.label"), "r") as f:
    header = f.readline()
    jlist = f.read().split("\n")


random.shuffle(jlist)
# split the list of 원천데이터 8:1:1
nj = len(jlist)
ntest = nvalid = int(0.1*nj)

jtest = jlist[:ntest]
jvalid = jlist[ntest:ntest+nvalid]
jtrain = jlist[ntest+nvalid:]


move_files(label_dir / Path(f"train.label"),
           jtrain,
           "train/")

move_files(label_dir / Path(f"test.label"),
           jtest,
           "test/")

move_files(label_dir / Path(f"valid.label"),
           jvalid,
           "valid/")

/home/hoseung/Work/NIA/data/raw/131/T1/Laptop/RGB/NIA22EYE_S1_131_T1_*_L_rgb_*_*_*.mp4
Found 87 videos
/home/hoseung/Work/NIA/data/raw/131/T1/Laptop/RGB/NIA22EYE_S1_131_T1_S02_L_rgb_A_T_T.mp4 done
/home/hoseung/Work/NIA/data/raw/131/T1/Laptop/RGB/NIA22EYE_S1_131_T1_S03_L_rgb_N_E_T.mp4 done
/home/hoseung/Work/NIA/data/raw/131/T1/Laptop/RGB/NIA22EYE_S1_131_T1_S03_L_rgb_S_E_T.mp4 done
/home/hoseung/Work/NIA/data/raw/131/T1/Laptop/RGB/NIA22EYE_S1_131_T1_S01_L_rgb_A_F_T.mp4 done
/home/hoseung/Work/NIA/data/raw/131/T1/Laptop/RGB/NIA22EYE_S1_131_T1_S01_L_rgb_N_D_T.mp4 done
/home/hoseung/Work/NIA/data/raw/131/T1/Laptop/RGB/NIA22EYE_S1_131_T1_S02_L_rgb_N_T_T.mp4 done
/home/hoseung/Work/NIA/data/raw/131/T1/Laptop/RGB/NIA22EYE_S1_131_T1_S03_L_rgb_S_D_T.mp4 done
/home/hoseung/Work/NIA/data/raw/131/T1/Laptop/RGB/NIA22EYE_S1_131_T1_S03_L_rgb_N_U_T.mp4 done
/home/hoseung/Work/NIA/data/raw/131/T1/Laptop/RGB/NIA22EYE_S1_131_T1_S02_L_rgb_A_E_T.mp4 done
/home/hoseung/Work/NIA/data/raw/131/T1/Laptop/RGB/N

/home/hoseung/Work/NIA/data/raw/131/T1/Laptop/RGB/NIA22EYE_S1_131_T1_S02_L_rgb_D_T_T.mp4 done
/home/hoseung/Work/NIA/data/raw/131/T1/Laptop/RGB/NIA22EYE_S1_131_T1_S03_L_rgb_D_C_T.mp4 done
/home/hoseung/Work/NIA/data/raw/131/T1/Laptop/RGB/NIA22EYE_S1_131_T1_S03_L_rgb_F_E_T.mp4 done
/home/hoseung/Work/NIA/data/raw/131/T1/Laptop/RGB/NIA22EYE_S1_131_T1_S03_L_rgb_F_F_T.mp4 done
/home/hoseung/Work/NIA/data/raw/131/T1/Laptop/RGB/NIA22EYE_S1_131_T1_S01_L_rgb_A_E_T.mp4 done
/home/hoseung/Work/NIA/data/raw/131/T1/Laptop/RGB/NIA22EYE_S1_131_T1_S01_L_rgb_F_T_T.mp4 done
/home/hoseung/Work/NIA/data/raw/131/T1/Laptop/RGB/NIA22EYE_S1_131_T1_S02_L_rgb_D_C_T.mp4 done
/home/hoseung/Work/NIA/data/raw/131/T1/Laptop/RGB/NIA22EYE_S1_131_T1_S01_L_rgb_S_D_T.mp4 done
/home/hoseung/Work/NIA/data/raw/131/T1/Laptop/RGB/NIA22EYE_S1_131_T1_S03_L_rgb_D_U_T.mp4 done
/home/hoseung/Work/NIA/data/raw/131/T1/Laptop/RGB/NIA22EYE_S1_131_T1_S01_L_rgb_F_F_T.mp4 done
/home/hoseung/Work/NIA/data/raw/131/T1/Laptop/RGB/NIA22EYE_S

/home/hoseung/Work/NIA/data/raw/131/T1/Monitor/RGB/NIA22EYE_S1_131_T1_S01_M_rgb_N_D_T.mp4 done
/home/hoseung/Work/NIA/data/raw/131/T1/Monitor/RGB/NIA22EYE_S1_131_T1_S03_M_rgb_D_U_T.mp4 done
/home/hoseung/Work/NIA/data/raw/131/T1/Monitor/RGB/NIA22EYE_S1_131_T1_S02_M_rgb_A_E_T.mp4 done
/home/hoseung/Work/NIA/data/raw/131/T1/Monitor/RGB/NIA22EYE_S1_131_T1_S01_M_rgb_S_D_T.mp4 done
/home/hoseung/Work/NIA/data/raw/131/T1/Monitor/RGB/NIA22EYE_S1_131_T1_S01_M_rgb_A_C_T.mp4 done
/home/hoseung/Work/NIA/data/raw/131/T1/Monitor/RGB/NIA22EYE_S1_131_T1_S01_M_rgb_F_S_T.mp4 done
/home/hoseung/Work/NIA/data/raw/131/T1/Monitor/RGB/NIA22EYE_S1_131_T1_S02_M_rgb_N_D_T.mp4 done
/home/hoseung/Work/NIA/data/raw/131/T1/Monitor/RGB/NIA22EYE_S1_131_T1_S02_M_rgb_F_C_T.mp4 done
/home/hoseung/Work/NIA/data/raw/131/T1/Monitor/RGB/NIA22EYE_S1_131_T1_S03_M_rgb_N_S_T.mp4 done
/home/hoseung/Work/NIA/data/raw/131/T1/Monitor/RGB/NIA22EYE_S1_131_T1_S02_M_rgb_N_E_T.mp4 done
/home/hoseung/Work/NIA/data/raw/131/T1/Monitor/RGB

something wrong /home/hoseung/Work/NIA/data/json/131/T1/VehicleLCD/json_rgb/NIA22EYE_S1_131_T1_S03_V_rgb_F_H_T_201.json
something wrong /home/hoseung/Work/NIA/data/json/131/T1/VehicleLCD/json_rgb/NIA22EYE_S1_131_T1_S03_V_rgb_F_H_T_202.json
something wrong /home/hoseung/Work/NIA/data/json/131/T1/VehicleLCD/json_rgb/NIA22EYE_S1_131_T1_S03_V_rgb_F_H_T_208.json
something wrong /home/hoseung/Work/NIA/data/json/131/T1/VehicleLCD/json_rgb/NIA22EYE_S1_131_T1_S03_V_rgb_F_H_T_221.json
something wrong /home/hoseung/Work/NIA/data/json/131/T1/VehicleLCD/json_rgb/NIA22EYE_S1_131_T1_S03_V_rgb_F_H_T_224.json
something wrong /home/hoseung/Work/NIA/data/json/131/T1/VehicleLCD/json_rgb/NIA22EYE_S1_131_T1_S03_V_rgb_F_H_T_231.json
something wrong /home/hoseung/Work/NIA/data/json/131/T1/VehicleLCD/json_rgb/NIA22EYE_S1_131_T1_S03_V_rgb_F_H_T_232.json
something wrong /home/hoseung/Work/NIA/data/json/131/T1/VehicleLCD/json_rgb/NIA22EYE_S1_131_T1_S03_V_rgb_F_H_T_258.json
something wrong /home/hoseung/Work/NIA/d

something wrong /home/hoseung/Work/NIA/data/json/131/T1/VehicleLCD/json_rgb/NIA22EYE_S1_131_T1_S03_V_rgb_F_E_T_092.json
something wrong /home/hoseung/Work/NIA/data/json/131/T1/VehicleLCD/json_rgb/NIA22EYE_S1_131_T1_S03_V_rgb_F_E_T_116.json
something wrong /home/hoseung/Work/NIA/data/json/131/T1/VehicleLCD/json_rgb/NIA22EYE_S1_131_T1_S03_V_rgb_F_E_T_125.json
something wrong /home/hoseung/Work/NIA/data/json/131/T1/VehicleLCD/json_rgb/NIA22EYE_S1_131_T1_S03_V_rgb_F_E_T_136.json
something wrong /home/hoseung/Work/NIA/data/json/131/T1/VehicleLCD/json_rgb/NIA22EYE_S1_131_T1_S03_V_rgb_F_E_T_171.json
something wrong /home/hoseung/Work/NIA/data/json/131/T1/VehicleLCD/json_rgb/NIA22EYE_S1_131_T1_S03_V_rgb_F_E_T_190.json
something wrong /home/hoseung/Work/NIA/data/json/131/T1/VehicleLCD/json_rgb/NIA22EYE_S1_131_T1_S03_V_rgb_F_E_T_207.json
something wrong /home/hoseung/Work/NIA/data/json/131/T1/VehicleLCD/json_rgb/NIA22EYE_S1_131_T1_S03_V_rgb_F_E_T_228.json
something wrong /home/hoseung/Work/NIA/d

...

ddddd /home/hoseung/Work/NIA/data/l2cs_data/Image/Face/002366.jpg
ddddd /home/hoseung/Work/NIA/data/l2cs_data/Image/Face/006383.jpg
ddddd /home/hoseung/Work/NIA/data/l2cs_data/Image/Face/000081.jpg
ddddd /home/hoseung/Work/NIA/data/l2cs_data/Image/Face/007375.jpg
ddddd /home/hoseung/Work/NIA/data/l2cs_data/Image/Face/006912.jpg
ddddd /home/hoseung/Work/NIA/data/l2cs_data/Image/Face/003263.jpg
ddddd /home/hoseung/Work/NIA/data/l2cs_data/Image/Face/007843.jpg
ddddd /home/hoseung/Work/NIA/data/l2cs_data/Image/Face/019250.jpg
ddddd /home/hoseung/Work/NIA/data/l2cs_data/Image/Face/011557.jpg
ddddd /home/hoseung/Work/NIA/data/l2cs_data/Image/Face/012806.jpg
ddddd /home/hoseung/Work/NIA/data/l2cs_data/Image/Face/020792.jpg
ddddd /home/hoseung/Work/NIA/data/l2cs_data/Image/Face/002329.jpg
ddddd /home/hoseung/Work/NIA/data/l2cs_data/Image/Face/016483.jpg
ddddd /home/hoseung/Work/NIA/data/l2cs_data/Image/Face/022406.jpg
ddddd /home/hoseung/Work/NIA/data/l2cs_data/Image/Face/010858.jpg
ddddd /hom

ddddd /home/hoseung/Work/NIA/data/l2cs_data/Image/Face/011909.jpg
ddddd /home/hoseung/Work/NIA/data/l2cs_data/Image/Face/020620.jpg
ddddd /home/hoseung/Work/NIA/data/l2cs_data/Image/Face/015948.jpg
ddddd /home/hoseung/Work/NIA/data/l2cs_data/Image/Face/015737.jpg
ddddd /home/hoseung/Work/NIA/data/l2cs_data/Image/Face/001648.jpg
ddddd /home/hoseung/Work/NIA/data/l2cs_data/Image/Face/012492.jpg
ddddd /home/hoseung/Work/NIA/data/l2cs_data/Image/Face/007734.jpg
ddddd /home/hoseung/Work/NIA/data/l2cs_data/Image/Face/008170.jpg
ddddd /home/hoseung/Work/NIA/data/l2cs_data/Image/Face/016121.jpg
ddddd /home/hoseung/Work/NIA/data/l2cs_data/Image/Face/004705.jpg
ddddd /home/hoseung/Work/NIA/data/l2cs_data/Image/Face/014509.jpg
ddddd /home/hoseung/Work/NIA/data/l2cs_data/Image/Face/012668.jpg
ddddd /home/hoseung/Work/NIA/data/l2cs_data/Image/Face/010956.jpg
ddddd /home/hoseung/Work/NIA/data/l2cs_data/Image/Face/022876.jpg
ddddd /home/hoseung/Work/NIA/data/l2cs_data/Image/Face/016652.jpg
ddddd /hom

ddddd /home/hoseung/Work/NIA/data/l2cs_data/Image/Face/004839.jpg
ddddd /home/hoseung/Work/NIA/data/l2cs_data/Image/Face/005944.jpg
ddddd /home/hoseung/Work/NIA/data/l2cs_data/Image/Face/022427.jpg
ddddd /home/hoseung/Work/NIA/data/l2cs_data/Image/Face/006773.jpg
ddddd /home/hoseung/Work/NIA/data/l2cs_data/Image/Face/021535.jpg
ddddd /home/hoseung/Work/NIA/data/l2cs_data/Image/Face/000386.jpg
ddddd /home/hoseung/Work/NIA/data/l2cs_data/Image/Face/010862.jpg
ddddd /home/hoseung/Work/NIA/data/l2cs_data/Image/Face/001615.jpg
ddddd /home/hoseung/Work/NIA/data/l2cs_data/Image/Face/013616.jpg
ddddd /home/hoseung/Work/NIA/data/l2cs_data/Image/Face/010833.jpg
ddddd /home/hoseung/Work/NIA/data/l2cs_data/Image/Face/018029.jpg
ddddd /home/hoseung/Work/NIA/data/l2cs_data/Image/Face/003103.jpg
ddddd /home/hoseung/Work/NIA/data/l2cs_data/Image/Face/018508.jpg
ddddd /home/hoseung/Work/NIA/data/l2cs_data/Image/Face/015114.jpg
ddddd /home/hoseung/Work/NIA/data/l2cs_data/Image/Face/000572.jpg
ddddd /hom

ddddd /home/hoseung/Work/NIA/data/l2cs_data/Image/Face/019734.jpg
ddddd /home/hoseung/Work/NIA/data/l2cs_data/Image/Face/019226.jpg
ddddd /home/hoseung/Work/NIA/data/l2cs_data/Image/Face/021281.jpg
ddddd /home/hoseung/Work/NIA/data/l2cs_data/Image/Face/000268.jpg
ddddd /home/hoseung/Work/NIA/data/l2cs_data/Image/Face/022697.jpg
ddddd /home/hoseung/Work/NIA/data/l2cs_data/Image/Face/012669.jpg
ddddd /home/hoseung/Work/NIA/data/l2cs_data/Image/Face/014201.jpg
ddddd /home/hoseung/Work/NIA/data/l2cs_data/Image/Face/009855.jpg
ddddd /home/hoseung/Work/NIA/data/l2cs_data/Image/Face/016775.jpg
ddddd /home/hoseung/Work/NIA/data/l2cs_data/Image/Face/005870.jpg
ddddd /home/hoseung/Work/NIA/data/l2cs_data/Image/Face/020232.jpg
ddddd /home/hoseung/Work/NIA/data/l2cs_data/Image/Face/000184.jpg
ddddd /home/hoseung/Work/NIA/data/l2cs_data/Image/Face/019648.jpg
ddddd /home/hoseung/Work/NIA/data/l2cs_data/Image/Face/011627.jpg
ddddd /home/hoseung/Work/NIA/data/l2cs_data/Image/Face/015812.jpg
ddddd /hom

Error: Cannot move a directory '/home/hoseung/Work/NIA/data/l2cs_data/Image' into itself '/home/hoseung/Work/NIA/data/l2cs_data/Image/valid'.